In [48]:
from numpy import linalg as LA
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import networkx as nx
import collections
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss
import warnings
from sklearn.svm import SVC
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.mixture import GaussianMixture
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from  sklearn.cluster import KMeans
from sklearn.metrics import roc_curve,auc,roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import  average_precision_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.metrics import classification_report, confusion_matrix


In [51]:
def model_performance_evaluation(y_test, y_pred):
    print("=== Confusion Matrix ===")
    print(confusion_matrix(y_test, y_pred))
    print('\n')
    print("=== Classification Report ===")
    print(classification_report(y_test, y_pred))
    print('\n')

In [34]:
df = pd.read_csv("usersdata.csv", delimiter = '\t', names = ["ID", "Gender", "Time", "Age", "Label"], index_col = "ID")

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [35]:
df

,Gender,Time,Age,Label
ID,,,,
1,M,0.9000,30,0
2,F,1.0000,20,0
3,M,0.1375,30,0
4,M,0.3875,20,0
5,M,0.0125,20,0
6,M,0.7125,20,0
7,M,0.9250,20,0
8,M,0.7875,20,0
9,M,0.0750,20,0


In [36]:
df["Label"].value_counts()

0    5270494
1     336953
Name: Label, dtype: int64

In [37]:
df.sample(frac = 0.01)["Label"].value_counts()

0    52640
1     3434
Name: Label, dtype: int64

In [38]:
# Assign weights to sample equal number of Spam and Ham
df["inv_value_count"] = 0
df["inv_value_count"][df.Label == 1] = 1/df["Label"].value_counts()[1]
df["inv_value_count"][df.Label == 0] = 1/df["Label"].value_counts()[0]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [39]:
# to sample equal number of spam/ham we add weight with inv_value_count columns
df_sample = df.sample(n = 50000, weights = 'inv_value_count').iloc[:, :4]

In [40]:
df_sample.head()

,Gender,Time,Age,Label
ID,,,,
1297266,F,0.6625,40,0
2966037,M,0.2750,30,1
2566815,M,0.0625,30,0
5183592,F,0.0000,20,1
83884,F,0.4250,20,0


In [41]:
df_sample["Label"].value_counts()

0    25457
1    24543
Name: Label, dtype: int64

equal number of spam/ham users

In [42]:
# transform categorical features into one-hot-encoding
df_sample_preproc = pd.get_dummies(data = df_sample, columns = ["Age", "Gender"])

In [43]:
df_sample_preproc.head(5)

,Time,Label,Age_10,Age_20,Age_30,Age_40,Age_50,Age_60,Age_70,Gender_F,Gender_M
ID,,,,,,,,,,,
1297266,0.6625,0,0,0,0,1,0,0,0,1,0
2966037,0.2750,1,0,0,1,0,0,0,0,0,1
2566815,0.0625,0,0,0,1,0,0,0,0,0,1
5183592,0.0000,1,0,1,0,0,0,0,0,1,0
83884,0.4250,0,0,1,0,0,0,0,0,1,0


In [44]:
Y_values = df_sample_preproc["Label"]
X_values = df_sample_preproc.drop(["Label"], axis = 1)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_values, Y_values, test_size=0.3)

## KNN

In [25]:
grid_params_knn = {
    'n_neighbors' : [3, 11, 59, 101, 501],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

In [27]:
gs_knn = GridSearchCV(KNeighborsClassifier(), grid_params_knn, verbose = 1, cv = 3, n_jobs = -1)
gs_results_knn = gs_knn.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.6min finished


In [28]:
print(gs_results_knn.best_score_)
print(gs_results_knn.best_estimator_)
print(gs_results_knn.best_params_)


0.7244857142857143
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=101, p=2,
           weights='uniform')
{'metric': 'euclidean', 'n_neighbors': 101, 'weights': 'uniform'}


### predict and test

In [52]:
knn_best_params_ = {'metric': 'euclidean', 'n_neighbors': 101, 'weights': 'uniform'}
knn = KNeighborsClassifier(**knn_best_params_)
knn.fit(X_train,y_train)
knn_predict = knn.predict(X_test)
model_performance_evaluation(y_test, knn_predict)

=== Confusion Matrix ===
[[6380 1225]
 [3067 4328]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.68      0.84      0.75      7605
           1       0.78      0.59      0.67      7395

   micro avg       0.71      0.71      0.71     15000
   macro avg       0.73      0.71      0.71     15000
weighted avg       0.73      0.71      0.71     15000





## Random Forest

In [17]:
grid_params_rf = {
    'bootstrap': [True],
    'max_depth': [2, 5, 10],
    'n_estimators': [10, 20, 50]
}
gs_rf = GridSearchCV(RandomForestClassifier(), grid_params_rf, verbose = 1, cv = 3, n_jobs = -1)
gs_results_rf = gs_rf.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   10.8s finished


In [18]:
print(gs_results_rf.best_score_)
print(gs_results_rf.best_estimator_)
print(gs_results_rf.best_params_)


0.7252571428571428
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
{'bootstrap': True, 'max_depth': 10, 'n_estimators': 20}


In [54]:
rf_best_params_ = {'bootstrap': True, 'max_depth': 10, 'n_estimators': 20}
rf = RandomForestClassifier(**rf_best_params_)
rf.fit(X_train,y_train)
rf_predict = rf.predict(X_test)
model_performance_evaluation(y_test, rf_predict)

=== Confusion Matrix ===
[[6389 1216]
 [3062 4333]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.68      0.84      0.75      7605
           1       0.78      0.59      0.67      7395

   micro avg       0.71      0.71      0.71     15000
   macro avg       0.73      0.71      0.71     15000
weighted avg       0.73      0.71      0.71     15000





## Gradient Boosted Decision Tree

In [19]:
from xgboost import XGBClassifier

In [20]:
xgb = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

In [22]:
grid_params_xgb = {
    'max_depth': [2, 5, 10],
    'n_estimators': [10, 20, 50, 100],
    'learning_rate': [0.1, 0.01, 0.05]
}

In [23]:
gs_xgb = GridSearchCV(xgb, grid_params_xgb, verbose = 1, cv = 3, n_jobs = -1)
gs_results_xgb = gs_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   45.8s finished


In [24]:
print(gs_results_xgb.best_score_)
print(gs_results_xgb.best_estimator_)
print(gs_results_xgb.best_params_)


0.7260857142857143
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


In [55]:
rf_best_params_ = {'bootstrap': True, 'max_depth': 10, 'n_estimators': 20}
rf = RandomForestClassifier(**rf_best_params_)
rf.fit(X_train,y_train)
rf_predict = rf.predict(X_test)
model_performance_evaluation(y_test, rf_predict)

=== Confusion Matrix ===
[[6401 1204]
 [3094 4301]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.67      0.84      0.75      7605
           1       0.78      0.58      0.67      7395

   micro avg       0.71      0.71      0.71     15000
   macro avg       0.73      0.71      0.71     15000
weighted avg       0.73      0.71      0.71     15000





## t-SNE

In [30]:
from sklearn.manifold import TSNE
X_tsne = TSNE(n_components=2, random_state=0).fit_transform(X_values) 


KeyboardInterrupt: 